In [1]:
# Ajustes del sistema y manejo de módulos personalizados
import sys
sys.path.append('..')

# Importación de módulos personalizados para funcionalidades específicas
from utils import evaluate_and_plot_model

ImportError: cannot import name 'evaluate_and_plot_model' from 'utils' (unknown location)

In [ ]:
import joblib

# Load RandomForest model
rf_model = joblib.load('random_forest_model.pkl')

# Load BaggingClassifier model
bagging_model = joblib.load('bagging_classifier_model.pkl')

# Load Gradient Boosting Classifier model
gb_model = joblib.load('gbc_model.pkl')

In [ ]:
# Load data
X_train = joblib.load('X_train.pkl')
X_test = joblib.load('X_test.pkl')
y_train = joblib.load('y_train.pkl')
y_test = joblib.load('y_test.pkl')

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)

In [ ]:
# Define the Selected Features for Each Model
rf_features = ['imc', 'total force rate', 'step rate', 'pronation excursion (mp->to)',
               'impact gs', 'contact ratio', 'max pronation velocity', 'power',
               'stance excursion (fs->mp)', 'vertical spring stiffness', 'braking gs',
               'peso', 'pronation excursion', 'footstrike type', 'stance excursion (mp->to)',
               'pace', 'step length', 'altura', 'vertical grf rate', 'edad', 'stride angle',
               'num calzado', 'shock', 'velocidad', 'tfe', 'fpi_total_d', 'hl', 'thomas tfl',
               'genu recurvatum', 'jack no reconstruye']

bagging_features = ['imc', 'total force rate', 'step rate', 'pronation excursion (mp->to)',
                    'contact ratio', 'footstrike type', 'vertical spring stiffness', 'power',
                    'max pronation velocity', 'pronation excursion', 'stance excursion (fs->mp)',
                    'braking gs', 'stance excursion (mp->to)', 'edad', 'step length', 'impact gs',
                    'pace', 'peso', 'stride angle', 'altura', 'shock', 'vertical grf rate',
                    'fpi_total_d', 'num calzado', 'velocidad', 'thomas tfl', 'tfe', 'lunge',
                    'arco aplanado', 'jack normal', 'jack no reconstruye', 'thomas psoas', 'tte',
                    'genu flexum', 'popliteo', 'genu recurvatum', 'm1 dfx', 'hav', 'pnca rp valgo',
                    'hl', 'm1 hipermovil', 'tfi', 'sexo', 'genu valgo', 'genu varo']

gb_features = ['imc', 'pronation excursion (mp->to)', 'vertical grf rate',
               'max pronation velocity', 'altura', 'pace', 'power', 'step rate',
               'stride angle', 'pronation excursion', 'fpi_total_d', 'peso', 'thomas psoas',
               'stance excursion (mp->to)', 'braking gs', 'step length', 'edad',
               'footstrike type', 'shock', 'jack no reconstruye', 'thomas tfl',
               'total force rate', 'hr', 'pnca rp valgo', 'stance excursion (fs->mp)',
               'num calzado', 'arco aplanado', 'tfe', 'vertical spring stiffness',
               'tibia vara proximal', 'rotula ascendida', 'contact ratio', 'impact gs', 'hl',
               'lunge', 'velocidad', 'hav', 'm5 hipermovil', 'ober friccion', 'jack normal',
               'm1 hipermovil', 'pronacion no disponible', 'popliteo', 'rotula convergente',
               'arco elevado']

In [ ]:
# Create Data Subsets for Each Model
X_train_rf = X_train[rf_features]
X_test_rf = X_test[rf_features]

X_train_bagging = X_train[bagging_features]
X_test_bagging = X_test[bagging_features]

X_train_gb = X_train[gb_features]
X_test_gb = X_test[gb_features]

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score

class ModelWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model, features):
        self.model = model
        self.features = features

    def fit(self, X, y):
        self.model.fit(X[self.features], y)
        return self

    def predict(self, X):
        return self.model.predict(X[self.features])

    def predict_proba(self, X):
        return self.model.predict_proba(X[self.features])

# Envolver los modelos con las características correspondientes
wrapped_rf = ModelWrapper(rf_model, rf_features)
wrapped_bagging = ModelWrapper(bagging_model, bagging_features)
wrapped_gb = ModelWrapper(gb_model, gb_features)

# Crear el VotingClassifier
voting_clf = VotingClassifier(
    estimators=[
        ('rf', wrapped_rf),
        ('bagging', wrapped_bagging),
        ('gb', wrapped_gb)
    ],
    voting='soft'  # 'soft' para usar las probabilidades de predicción
)

# Ajustar el VotingClassifier con las características comunes (en este caso, usamos todas las características)
voting_clf.fit(X_train, y_train)

# Hacer predicciones y evaluar el modelo
y_pred_voting = voting_clf.predict(X_test)
print(classification_report(y_test, y_pred_voting))

# Calcular y mostrar el AUC-ROC
y_prob_voting = voting_clf.predict_proba(X_test)[:, 1]
auc_roc_voting = roc_auc_score(y_test, y_prob_voting)
print(f"AUC-ROC: {auc_roc_voting:.4f}")

In [ ]:
print("Train: ",voting_clf .score(X_train,y_train))
print("Test: ",voting_clf .score(X_test,y_test))

In [ ]:
class_names = ['no rodilla', 'rodilla']
evaluate_and_plot_model(voting_clf, X_test, y_test, class_names)